# Analysis of Energy Efficiency using Kepler.gl


We analyse energy efficiency in Wales and compare it to the Index of Multiple Deprivation (IMD). For visualisation, we use [kepler.gl](https://kepler.gl/).

### Imports

In [1]:
import yaml
import pandas as pd
import numpy as np
from keplergl import KeplerGl

from epc_data_analysis.getters import epc_data, util_data
from epc_data_analysis.pipeline import feature_engineering

### Load Config File

In [2]:
with open("Kepler/tenure_type_correct_colors_IMD_config.txt", "r") as infile:
    config = infile.read()
    config = yaml.load(config, Loader=yaml.FullLoader)

### Load EPC data 

Only load necessary features and subset to keep Kepler from crashing.

In [3]:
# Set features of interest
features_of_interest = ["TENURE", "CURRENT_ENERGY_RATING", "POSTCODE", "POTENTIAL_ENERGY_RATING"]

# Load Wales EPC data
epc_df = epc_data.load_EPC_data(subset="Wales", usecols=features_of_interest, low_memory=False)
location_df = util_data.get_location_data()

/Users/juliasuter/opt/anaconda3/envs/EPC_data_analysis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


### Load Location Data and additional EPC features

And remove uncessary features afterwards (POSTCODE, POTENTIAL_ENERGY_RATING).

Remove samples with invalid CURRENT_ENERGY_RATING.

In [4]:
# Merge with location data 
epc_df = util_data.merge_dataframes(epc_df, location_df, "POSTCODE")

# Get additional features
epc_df = feature_engineering.get_new_EPC_rating_features(epc_df)

# Remove unnecessary features
epc_df = epc_df[["TENURE", "CURRENT_ENERGY_RATING",
                 "LATITUDE", "LONGITUDE", "ENERGY_RATING_CAT"]]

epc_df = epc_df.drop(
    epc_df[epc_df.CURRENT_ENERGY_RATING == "INVALID!"].index
)
epc_df.head()

,TENURE,CURRENT_ENERGY_RATING,LATITUDE,LONGITUDE,ENERGY_RATING_CAT
0,owner-occupied,C,51.830316,-3.014073,C-D
1,owner-occupied,D,51.830316,-3.014073,C-D
2,rental (social),C,51.830316,-3.014073,C-D
3,owner-occupied,D,51.830316,-3.014073,C-D
4,owner-occupied,D,51.830316,-3.014073,C-D


### Load Wales IMD data

In [5]:
# Load Wales IMD data
wimd_df = util_data.get_WIMD_data()
wimd_df = wimd_df[['WIMD Decile', 'LATITUDE', 'LONGITUDE', 'WIMD Score']]
wimd_df.head()

,WIMD Decile,LATITUDE,LONGITUDE,WIMD Score
0,6,51.479111,-3.177999,17.1
1,6,51.478699,-3.177830,17.1
2,6,51.478065,-3.177410,17.1
3,6,51.478484,-3.176987,17.1
4,6,51.478813,-3.177228,17.1


### Load Data in Kepler Visualisation

Load 4 different layers:

    - rental (social)
    - rental (private)
    - owner-occupied
    - IMD

In [6]:
tenure_type_map = KeplerGl(height=500, config=config)

tenure_type_map.add_data(
    data=epc_df.loc[epc_df["TENURE"] == "rental (social)"], name="social"
)
tenure_type_map.add_data(
    data=epc_df.loc[epc_df["TENURE"] == "rental (private)"], name="private"
)
tenure_type_map.add_data(
    data=epc_df.loc[epc_df["TENURE"] == "owner-occupied"], name="owner-occupied"
)
tenure_type_map.add_data(data=wimd_df, name="WIMD")

tenure_type_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


### Save Map and Config File

In [ ]:
tag = "_x"

tenure_type_map.save_to_html(
    file_name="../../../outputs/data/Wales/Kepler/Wales_EPC_IMD.html"
)
with open("Kepler/tenure_type_correct_colors_IMD_config{}.txt".format(tag), "w") as outfile:
    outfile.write(str(tenure_type_map.config))